In [1]:
import numpy as np
import matplotlib.pyplot as plt

from dataset_tools import image_tools as it
from dataset_tools import mhio
from minihip import inference_tools as inf


from scipy.ndimage import gaussian_laplace
from scipy.ndimage import gaussian_filter
from scipy.ndimage import sobel
import scipy.ndimage as ndimage
import os

"""
Author - l.millward@qmul.ac.uk
for MoEDAL experiment
"""
folders_exposed = [('exposed', 'dirty_s', 1, 250), ('exposed_eval', 'dirty_s', 251, 500), 
                       ('exposed_reverse', 'dirty_r_s', 1, 250), ('exposed_reverse_eval', 'dirty_r_s', 251, 500)]
    
folders_clean = [('clean','clean1_s', 1, 250), ('clean_2', 'clean2_s', 1, 250), 
                     ('clean_2_eval', 'clean2_s', 251, 500), ('clean_eval', 'clean1_s', 251, 500), 
                     ('clean_r', 'clean1_r_s', 1, 250), ('clean_r_eval', 'clean1_r_s', 251, 500)]
print("Complete")

Complete


In [2]:
def test_inf(ic=None,id=None):
    """
    1) Create ensemble of models
    """
    #FCNN = inf.Ensemble('./md{0}.h5',10,(480,640,8))
    FCNN = inf.Ensemble('./fold-{0}.h5', 9, (480,640,8))
    slide_path = '/Users/adamkhan/Documents/DPALARGEDATA/all_data_png'
    
    
    # To get the file path of each image image, to change the selected 
    # folders change the relevant folders in variable folders

    heatmap_output = {}
    
    """
    2) Generate inference predictions for each microscope-slide
    """
    #folders_clean=[]
    #for folder,slide_name, minimum, maximum in folders_clean:
        #outputs = []
        #newpath = os.path.join(slide_path, folder)
        #if ic:
            #minimum = minimum + ic
            #maximum = minimum + 1
        #for foil_number in range(minimum, maximum):
            #print("  Clean:",folder," : ",foil_number)
            #filename = newpath+"/{0}"+ str(foil_number) +"_{2}.png"
            # print(folder, slide_name, newpath, filename)
            #slide = it.Slide(slide_name, foil_number, filename, flag = 'b')
            #outputs.append((foil_number, slide_name, FCNN.predict_slide(slide.raw)))
        #heatmap_output[folder] = outputs
        # print('Ran A')
    
    for folder,slide_name, minimum, maximum in folders_exposed:
        outputs = []
        newpath = os.path.join(slide_path, folder)
        # print(folder, slide_name, newpath)
        if id:
            minimum = minimum + id
            maximum = minimum + 1

        for foil_number in range(minimum, maximum):
            print("Ran B")
            print("Exposed: ",folder," : ", foil_number)
            filename = newpath+"/{0}"+ str(foil_number) +"_{2}.png"
            slide = it.Slide(slide_name, foil_number, filename)
            # print("The slide array is:", slide)
            p = FCNN.predict_slide(slide.raw)
            print('prediction: ', p)
            print(p.shape)
            outputs.append((foil_number, slide_name, p))
        # heatmap_output[folder] = FCNN.predict_slide(slide.raw)
        # heatmap_output[foil_number] = FCNN.predict_slide(slide.raw)
        heatmap_output[folder] = outputs
    print("The Keys Are",heatmap_output.keys())
            
    # foil = 'd_s'




    """
    3) Output is a dictionary, key i = slide / image number
       each entry is a (h,w,c) array with each channel c representing the heatmap for
       the c'th model in the ensemble - ie (60x80x5)
    """
    mhio.pickle_dump(heatmap_output,'./HeatMaps/heatmap_output')


print("Complete")

Complete


In [3]:
test_inf(1,1)

Original model: ./fold-0.h5
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 32)        2336      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 7, 7, 32)          9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_7 (Conv2D)         

Original model: ./fold-3.h5
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 28, 28, 32)        4128      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 14, 14, 32)        16416     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 7, 7, 32)          16416     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_19 (Conv2D)        

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 3 dimension(s)

In [ ]:
def scale(x,y,ratio=80./640.):
    """
    Take a list of x,y coordinates and rescale
    used to change between 60x80 heatmaps and 480x640 images
    """
    return [ X*ratio for X in x], [ Y*ratio for Y in y]


def process_heatmap(Z):
    """
    Take (60,80,5) heatmap array for i'th slide and produce a set of predicted x,y etch-pit locations.
    The locations predicted depend on the method used for deriving x,y coordinates
    """

    """
    Calc. average image, and gaussian laplace (of each channel) by default
    """
    print('heatmap shape:',Z.shape)
    zav = np.average(Z,axis=2)
    [ it.plot(Z[:,:,j]) for j in range(0,5)]
    it.plot(zav)    
    gauss = [gaussian_laplace(Z[:,:,j],sigma = 1.0) for j in range(0,5)]
    #it.plot(zav)

    #x,y = it.coord_list(im>th)
    #it.plot_coords(x,y,(im>th))

    def default_opt(Z,thresh=0.1):
        """
        Average of gaussain laplace functions, w. threshold of 0.1
        """
        gs = sum(gauss) # average the gaussians
        return it.coord_list(-gs>thresh)
    
    def opt2(Z,thresh=0.1):
        """
        No ensembling / average, just gaussian laplace for first model in ensemble
        """
        it.plot(-gauss[1]>0.1) ## without mask
        return it.coord_list(-gauss[1]>0.1)

    #X,Y = default_opt(Z)
    result = opt2(Z)
    # print("Result is :",result)
    if result.size==0:
        return ([],[])
    X,Y = result
    X,Y = scale(X,Y,640./80.)
    return X,Y


def predict_locations(path='./HeatMaps/heatmap_output',fixedi=None):
    """
    Load predicted heatmaps, key corresponds to predictions for a slide 'i'
    """
    
    slide_path = '/Users/adamkhan/Documents/DPALARGEDATA/all_data_png'
    
    hmaps = mhio.pickle_load(path)
    # print(hmaps)
    for folder,v in hmaps.items():
        for foil_number,slide_name,output in v:
            x,y = process_heatmap(output)        
            newpath = os.path.join(slide_path, folder)
            filename = newpath+"/{0}{1}_{2}.png"
            s = it.Slide(slide_name, foil_number, filename)
            print("You are looknig at folder:",folder,slide_name,foil_number)
            it.plot_coords(x,y,s.b)

        # s = it.Slide('dirty_s',i,'/home/millward/Moedal_data/febdat/png_nov/{0:}{1:}_{2:}.png','b')
        # it.plot_coords(x,y,s.b)
        
        #if len(folder) <= 0: continue 
        #print("Your folder looks like this :",hmaps[folder])
        #x,y = process_heatmap(hmaps[folder])
        #for folder,slide_name, minimum, maximum in folders_clean:
            #newpath = os.path.join(slide_path, folder)
            #for foil_number in range(minimum, maximum):
                #filename = newpath+"/{0}"+ str(foil_number) +"_{2}.png"
                #s = it.Slide(slide_name, foil_number, filename)
                #it.plot_coords(x,y,s.b)
                
        
        # s = it.Slide('dirty_s',i,'/home/millward/Moedal_data/febdat/png_nov/{0:}{1:}_{2:}.png','b')
        # it.plot_coords(x,y,s.b)
        
        
        # print(folder)
        # print("The Maps Are",hmaps[folder])
        # if len(hmaps[folder]) == 0: continue
        # print(hmaps[folder])
        # for name,foil,output in hmaps[folder]:
            # print(outputs)
            # x,y = process_heatmap(output)
            # print(name, foil)
            # s = it.Slide(name,foil,'/Users/adamkhan/Documents/DPALARGEDATA/all_data_png/'+folder+'/{0:}{1:}_{2:}.png','b')
            # it.plot_coords(x,y,s.b)
        #it.plot_coords(x,y,s.b,save='save images here')
        
print("Complete")

In [ ]:
predict_locations()